In [9]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier

In [10]:
data=pd.read_excel('dataset/Bank_Personal_Loan_Modelling.xlsx')
data.head()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [11]:
dataset=data.drop(['ID','ZIP Code'],axis=1)
dataset.head()

,Age,Experience,Income,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,25,1,49,4,1.6,1,0,0,1,0,0,0
1,45,19,34,3,1.5,1,0,0,1,0,0,0
2,39,15,11,1,1.0,1,0,0,0,0,0,0
3,35,9,100,1,2.7,2,0,0,0,0,0,0
4,35,8,45,4,1.0,2,0,0,0,0,0,1


In [12]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 12 columns):
Age                   5000 non-null int64
Experience            5000 non-null int64
Income                5000 non-null int64
Family                5000 non-null int64
CCAvg                 5000 non-null float64
Education             5000 non-null int64
Mortgage              5000 non-null int64
Personal Loan         5000 non-null int64
Securities Account    5000 non-null int64
CD Account            5000 non-null int64
Online                5000 non-null int64
CreditCard            5000 non-null int64
dtypes: float64(1), int64(11)
memory usage: 468.8 KB


In [13]:
dataset.shape

(5000, 12)

In [14]:
dataset.isnull().sum()

Age                   0
Experience            0
Income                0
Family                0
CCAvg                 0
Education             0
Mortgage              0
Personal Loan         0
Securities Account    0
CD Account            0
Online                0
CreditCard            0
dtype: int64

In [15]:
dataset.columns

Index(['Age', 'Experience', 'Income', 'Family', 'CCAvg', 'Education',
       'Mortgage', 'Personal Loan', 'Securities Account', 'CD Account',
       'Online', 'CreditCard'],
      dtype='object')

In [16]:
rf_model=RandomForestClassifier(n_estimators=1000,max_features=2,oob_score=True)

In [17]:
features=['Age', 'Experience', 'Income','Family', 'CCAvg',
       'Education', 'Mortgage', 'Securities Account',
       'CD Account', 'Online', 'CreditCard']

In [18]:
rf_model.fit(X=data[features],y=data["Personal Loan"])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=2, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
            oob_score=True, random_state=None, verbose=0, warm_start=False)

In [19]:
print("Accuracy is:",rf_model.oob_score_)

Accuracy is: 0.9876


In [20]:
for feature,imp in zip(features,rf_model.feature_importances_):
    print(feature,imp);

Age 0.0447810822454
Experience 0.0442432151412
Income 0.340574216496
Family 0.0967341434105
CCAvg 0.184250723466
Education 0.166046050824
Mortgage 0.0442559635686
Securities Account 0.00561814348146
CD Account 0.0552199806583
Online 0.00827174073137
CreditCard 0.010004739977


Clearly Income,CCAvg,Education are most important features and decision tree will be created using these features only.

# Decision Tree

In [21]:
from sklearn import tree
predictors=dataset[['Income','CCAvg','Education']]
treemodel=tree.DecisionTreeClassifier(max_depth=8)
treemodel.fit(X=predictors,y=dataset['Personal Loan'])

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=8,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [22]:
with open("Personal_Loan_Decision_Tree.dot",'w')as f:
    f=tree.export_graphviz(treemodel,feature_names=["Income","CCAvg","Education"],out_file=f);

In [23]:
treemodel.score(X=predictors,y=dataset["Personal Loan"])

0.98160000000000003

# Leaf nodes inferences

1) 3629 customers having income less than 106.5 dollars and CCavg less than 2.95 dollars did not get loan.

2) 314 customers having income greater than 116.5 dollars with education level of either Graduate or Advanced/Professional got the loan.

3) 3 customers having income greater than 106.5 dollars and less than 113.5 with education level of either Graduate or Advanced/Professional and CCavg less than 0.3 dollars didn't get the loan.

4) 22 customers having income greater than 109.5 dollars and less than 113.5 with education level of Undergraduate or and CCavg greater than 0.3 dollars and less than 2.95 dollars didn't get the loan.

5) 60 customers having income less than 81.5 dollars with CCavg greater than 3.55 dollars didn't get the loan.

6) 4 customers having income less than 82.5 dollars bnt greater than 81.5 dollars with CCavg greater than 3.75 dollars didn't get the loan.